In [30]:
#Manejo de datos
import pandas as pd 
#Manejo de tensores 
import numpy as np
#Optimizacion y validacion
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
#Baggin
from sklearn.ensemble import BaggingClassifier
#Modelos lineales 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
#Modelos no lineales
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier, RandomForestClassifier 
from xgboost import XGBRFClassifier
#Metricas
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
#Preprocesadores
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
#Pipline 
from sklearn.pipeline import Pipeline
#Resampleo
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [2]:
#Cargar los datos 
df = pd.read_csv('../data/student_new.csv')
#Visualizarlos para comprobar
df.head(2)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,aprueba
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,0


# **Preparación de variables**

In [3]:
#Seleccion de las variables y separacion de sets para validar luego 
X = df.drop('aprueba', axis=1)
y = df['aprueba']

#Separacion
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

#Para modelos lineales 
X_train_lineal = X_train[['G1', 'G2']]
X_test_lineal = X_test[['G1', 'G2']]

In [4]:
#Datos resampleados 
"""---------------------------------------------------Lineales---------------------------------------------------------------"""
resampler_lineal = SMOTE(random_state=42)
X_train_smote, Y_train_smote = resampler_lineal.fit_resample(X_train_lineal, Y_train)
"""---------------------------------------------------No Lineales---------------------------------------------------------------"""
resampler_no_lineal = RandomOverSampler(random_state=42)
X_train_over, Y_train_over = resampler_no_lineal.fit_resample(X_train, Y_train)

In [5]:
print('Distribucion de Y_train_smote')
print(Y_train_smote.value_counts())
print(f'Total: {Y_train_smote.value_counts()[0] + Y_train_smote.value_counts()[1]}')
print('=' * 70)
print('Distribucion de Y_train_over')
print(Y_train_over.value_counts())
print(f'Total: {Y_train_over.value_counts()[0] + Y_train_over.value_counts()[1]}')

Distribucion de Y_train_smote
aprueba
0    212
1    212
Name: count, dtype: int64
Total: 424
Distribucion de Y_train_over
aprueba
0    212
1    212
Name: count, dtype: int64
Total: 424


In [6]:
#Varibles numericas y categoricas
numericas = X.select_dtypes(include=[np.number]).columns
categoricas = X.select_dtypes(exclude=[np.number]).columns

In [7]:
#Crear transformador para las variables categoricas 
transformer = ColumnTransformer(transformers=[
    ('num', 'passthrough', numericas),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categoricas)
])

# **Creación de modelos y funciones entrenamiento**

In [8]:
"""---------------------------------------------------Lineales---------------------------------------------------------------"""
modelos_lineales = {
    'baggin_logistic': BaggingClassifier(LogisticRegression(penalty='l1', solver='liblinear', C=1.0, max_iter=1000, random_state=42)),
    'LogisticRegression': LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, max_iter=1000,random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier()
}
"""---------------------------------------------------No Lineales---------------------------------------------------------------"""
modelos_no_lineales = {
    'DecisionTreeClassifier':DecisionTreeClassifier(max_depth=2, random_state=42),
    'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=5, random_state=42, splitter='best'),
    'RandomForestClassifier': RandomForestClassifier(random_state=42, n_estimators=50, max_depth=5),
    'GradientBoostingClassifier': GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=42),
    'XGBRFClassifier': XGBRFClassifier(n_estimators=100, learning_rate=1.0, subsample=0.8, random_state=42),
    'AdaBoostClassifier': AdaBoostClassifier(estimator=DecisionTreeClassifier(), learning_rate=1.0, random_state=42)
}

In [ ]:
def funcion_entrenamiento(datos_train:list, datos_test:list, ln_models: dict, unl_models:dict):
    """
    En esta funcion se encarga de entrenar los modelos de manera iterativa realizando predicciones y metricas
    para poder evaluar los modelos. Itera por cada uno de los modelos creando un pipeline que pueda tener 
    el preprocesador y el modelo. Finalmente se entrena y realiza las pruebas para las metricas guardandolas
    en un diccionario con el nombre del modelo y los resultados en una lista. Hace eso por cada iteracion
    y con los dos tipos de modelos , lieales y no lineales

    Parametros: 
    datos_train: Una lista que devuelve los datos para entrenar tanto lineales como no lineales 
    datos_test: Una lista que devuelde datos de testeo lineales y no lineales 
    ln_models:Devuelve un diccionario con modelos lineales
    unl_models:Devuelve un diccionario con modelos no lineales

    retur: dos diccionarios con los resultados lineales y no lineales
    """
    
    #Resultados obtenidos
    resultados_lineales = {}
    resultados_unlineales = {}

    #---------------------Para modelos lineales -----------------------
    print('='*70)
    print('🔴 Entrenando los modelos lineales...')
    print('='*70)
    for nombre, modelo in ln_models.items():
        print(f'Modelo {nombre} ha comenzado...')
        pipline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', modelo)
        ])

        pipline.fit(datos_train[0], datos_train[1])
        pred = pipline.predict(datos_test[0])
        report, matrix_conf, acc =  classification_report(datos_test[2], pred), confusion_matrix(datos_test[2], pred), accuracy_score(datos_test[2], pred)
        resultados_lineales[nombre] = [report, matrix_conf, acc]
        print(f'Modelo {nombre} ha finalizado')
        print('=-'*70)

    #---------------------Para modelos no lineales -----------------------
    print('='*70)
    print('🔵 Entrenando modelos no lineales...')
    print('='*70)

    for nombre, modelo in unl_models.items():
        print(f'Modelo {nombre} ha comenzado...')
        pipline = Pipeline([
            ('preprocesador', transformer),
            ('model', modelo)
        ])

        pipline.fit(datos_train[2], datos_train[1])
        pred = pipline.predict(datos_test[1])
        report, matrix_conf, acc = classification_report(datos_test[2], pred), confusion_matrix(datos_test[2], pred), accuracy_score(datos_test[2], pred)
        resultados_unlineales[nombre] = [report, matrix_conf, acc]
        print(f'Modelo {nombre} ha finalizado.')
        print('=-'*70)

    return [resultados_lineales, resultados_unlineales]

In [ ]:
#Listas con datos de entrenamiento y testeo para la funcion
lista_train = [X_train_smote, Y_train_smote, X_train_over]
lista_test = [X_test_lineal, X_test, Y_test]
resultados = funcion_entrenamiento(lista_train, lista_test, modelos_lineales, modelos_no_lineales)

🔴 Entrenando los modelos lineales...
Modelo baggin_logistic ha comenzado...
Modelo baggin_logistic ha finalizado
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Modelo LogisticRegression ha comenzado...
Modelo LogisticRegression ha finalizado
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Modelo KNeighborsClassifier ha comenzado...
Modelo KNeighborsClassifier ha finalizado
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
🔵 Entrenando modelos no lineales...
Modelo DecisionTreeClassifier ha comenzado...
Modelo DecisionTreeClassifier ha finalizado.
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Modelo RandomForestClassifie

# **Funcion de evaluacion**

In [ ]:
modelos = ['🔴lineales', '🔵no lineales']
def eval_fun(resultados, mostrar_info=None):
    """
    Esta funcion se encarga de revisar los diccionarios con resultados de la funcion que se encarga de entrenar 
    para poder guardarl los modelos superiores a 0.88 de accuracy  guarda los mismos parametros en un diccionario pero solamente
    los que pasan por cierto valor
    
    Parametros: 
    resultados: devuelve una lista con diccionarios para evaluar
    mostrar_info: se encarga de mostrar el contenido o no de los resultados 

    return un diccionario con los mejores modelos evaluados
    """
    mejores_modelos_accbased = {}
    for i, v in enumerate(resultados):
        if mostrar_info == True:
            print('='*50)
            print(f'Modelos: {modelos[i]}')
            print('='*50)
        for k, v in v.items():
            if mostrar_info == True:
                print('|'*50)
                print(f'Metricas para el modelo {k}')
                print(f'Reporte general:\n {v[0]}')
                print(f'Matriz de confusion {v[1]}')
            if v[2] >= 0.88:
                mejores_modelos_accbased[k] = [v[0], v[1], v[2]]
                
    return mejores_modelos_accbased
        

In [17]:
#Extraer l informacion de lo mejores modelos para tunerlos
mejores_modelos = eval_fun(resultados, mostrar_info=True)
print(f'Mejores modelos {mejores_modelos.keys()}')

Modelos: 🔴lineales
||||||||||||||||||||||||||||||||||||||||||||||||||
Metricas para el modelo baggin_logistic
Reporte general:
               precision    recall  f1-score   support

           0       0.74      0.96      0.83        26
           1       0.98      0.83      0.90        53

    accuracy                           0.87        79
   macro avg       0.86      0.90      0.87        79
weighted avg       0.90      0.87      0.88        79

Matriz de confusion [[25  1]
 [ 9 44]]
||||||||||||||||||||||||||||||||||||||||||||||||||
Metricas para el modelo LogisticRegression
Reporte general:
               precision    recall  f1-score   support

           0       0.68      0.96      0.79        26
           1       0.98      0.77      0.86        53

    accuracy                           0.84        79
   macro avg       0.83      0.87      0.83        79
weighted avg       0.88      0.84      0.84        79

Matriz de confusion [[25  1]
 [12 41]]
||||||||||||||||||||||||||||

# **Función de busqeuda de parametros**

In [13]:
#Piplines a realizar la busqueda
pipline_kn = Pipeline([('sacaler', StandardScaler()), ('model_kn', KNeighborsClassifier())])
pipline_grad = Pipeline([('proprocesador', transformer), ('model_grad', GradientBoostingClassifier(random_state=42))])
#Plantillas pra los Buscadores
plantillas = [{'KNeighborsClassifier':{
    'model_kn__n_neighbors': [3, 5, 7, 9, 11, 13, 15],         # número de vecinos
    'model_kn__weights': ['uniform', 'distance'],    # tipo de ponderación
    'model_kn__metric': ['euclidean', 'manhattan', 'minkowski'], # métrica de distancia
    'model_kn__p': [1, 2]                            # 1=Manhattan, 2=Euclidiana
}}, {'GradientBoostingClassifier':{
    'model_grad__n_estimators': [50, 100, 150, 200],     # número de árboles
    'model_grad__learning_rate': [0.01, 0.05, 0.1],  # tasa de aprendizaje
    'model_grad__max_depth': [2, 3, 4, 5],           # profundidad máxima
    'model_grad__min_samples_split': [2, 4, 6],  # mínimo para dividir nodo
    'model_grad__min_samples_leaf': [1, 2, 3],    # mínimo por hoja
    'model_grad__subsample': [0.7, 0.8, 1.0],       # muestreo del dataset
    'model_grad__max_features': ['sqrt', 'log2'] # características por árbol
}}]

In [ ]:
def busquedahyper(x_train_l,  x_train_n, y_train_l, y_train_n, modelos:list, verbose=False):
    """
    Obtiene los mejores modelos y se encarga de realizar una busqueda de hyper parametros
    de manera simultanea iterando por cada uno de ellos accediendo tambien a un diccionario
    con los modelos para poder utilizarlos en el buscador. Filtra para que se ajuste a los datos
    dependiendo el modelo que se tiene para que sea entrenado con datos lineales o no lineales

    Parametros: 
    x_train_l,  x_train_n, y_train_l, y_train_n: Son los datos de entrenamientos lineales y no lineales de los mehjores modelos
    modelos: una lista con diccionarios de los hyperparemtros a combinar 
    vebose:  booleano que se encarga de activar monitoreo(mensajes) durante el entrenamiento para poder ver como va (opcionalidad)

    return: Se encarga de devolverme una lista con dos lista con diccionarios los cuales tienen el nombre del modelo con sus 
    mejores parametros y mejor metrica de evaluacion
    """
    mejores_params = []
    mejores_resultados = []
    lineales = ['KNeighborsClassifier']
    pipelines = {
        'KNeighborsClassifier':pipline_kn,
        'GradientBoostingClassifier': pipline_grad
    }
    
    for i in modelos:
        for k, v in i.items():
            print('-'*50)
            print(f'Iniciando busqueda para: {k}')
            
            pipe = pipelines[k]

            search = GridSearchCV(
                estimator=pipe,
                param_grid=v,
                scoring='f1',
                cv=5,
                verbose=1 if verbose else 0
            )
            if k == lineales:
                search.fit(x_train_l, y_train_l)
            else:
                search.fit(x_train_n, y_train_n)
            
            mejores_params.append({k: search.best_params_})
            mejores_resultados.append({k: search.best_score_})

            print('-'*50)
            print(f'Finalizando busqueda para: {k}')

    return [mejores_params, mejores_resultados]
                

# **Seleccion del mejor modelo y validación** 

In [ ]:
#Instanciar los resultados 
resultados = busquedahyper(X_train_smote, X_train_over, Y_train_smote, Y_train_over, plantillas)

||||||||||||||||||||||||||||||||||||||||||||||||||
Iniciando busqueda para: KNeighborsClassifier
||||||||||||||||||||||||||||||||||||||||||||||||||
||||||||||||||||||||||||||||||||||||||||||||||||||
Iniciando busqueda para: GradientBoostingClassifier
||||||||||||||||||||||||||||||||||||||||||||||||||


In [ ]:
#Revisar los resultados de los parametros y metricas obetenidas
resultados

[[{'KNeighborsClassifier': {'model_kn__metric': 'euclidean',
    'model_kn__n_neighbors': 15,
    'model_kn__p': 1,
    'model_kn__weights': 'uniform'}},
  {'GradientBoostingClassifier': {'model_grad__learning_rate': 0.1,
    'model_grad__max_depth': 4,
    'model_grad__max_features': 'sqrt',
    'model_grad__min_samples_leaf': 3,
    'model_grad__min_samples_split': 2,
    'model_grad__n_estimators': 200,
    'model_grad__subsample': 1.0}}],
 [{'KNeighborsClassifier': np.float64(0.9319143514002721)},
  {'GradientBoostingClassifier': np.float64(0.963112080275148)}]]

In [ ]:
#Probar pipline con los parametros sugeridos
pipline_kn = Pipeline([('preprocesador', StandardScaler()), ('model', KNeighborsClassifier(n_neighbors=15, p=1, weights='uniform'))])
#Prediccion para el modelo
pipline_kn.fit(X_train_smote, Y_train_smote)
pred = pipline_kn.predict(X_test_lineal)
#Metricas para test
report, cf, acc = classification_report(Y_test, pred), confusion_matrix(Y_test, pred), accuracy_score(Y_test, pred)
print(f'Para TEST:\n\n Reporte general: \n\n {report} \n\n Matriz de confusión: \n\n {cf} \n\n Accuracy: \n\n {acc} \n\n')
#Metricas para train
pred = pipline_kn.predict(X_train_smote)
report, cf, acc = classification_report(Y_train_smote, pred), confusion_matrix(Y_train_smote, pred), accuracy_score(Y_train_smote, pred)
print(f'Para TRAIN: \n\n Reporte general: \n\n {report} \n\n Matriz de confusión: \n\n {cf} \n\n Accuracy: \n\n {acc}')

Para TEST:

 Reporte general: 

               precision    recall  f1-score   support

           0       0.74      0.96      0.83        26
           1       0.98      0.83      0.90        53

    accuracy                           0.87        79
   macro avg       0.86      0.90      0.87        79
weighted avg       0.90      0.87      0.88        79
 

 Matriz de confusión: 

 [[25  1]
 [ 9 44]] 

 Accuracy: 

 0.8734177215189873 


Para TRAIN: 

 Reporte general: 

               precision    recall  f1-score   support

           0       0.92      0.95      0.94       212
           1       0.95      0.92      0.94       212

    accuracy                           0.94       424
   macro avg       0.94      0.94      0.94       424
weighted avg       0.94      0.94      0.94       424
 

 Matriz de confusión: 

 [[202  10]
 [ 17 195]] 

 Accuracy: 

 0.9363207547169812


In [ ]:
#Probar modelo con los parametros sugeridos 
pipline_grad = Pipeline([('preprocesador', transformer), ('model', GradientBoostingClassifier(learning_rate=0.01,max_depth=4, max_features='sqrt', min_samples_leaf=3, min_samples_split=2, n_estimators=100, subsample=1.0))])
#Prediccion para el modelo
pipline_grad.fit(X_train_over, Y_train_over)
pred = pipline_grad.predict(X_test)
#Metricas para test
report, cf, acc = classification_report(Y_test, pred), confusion_matrix(Y_test, pred), accuracy_score(Y_test, pred)
print(f'Para TEST:\n\n Reporte general: \n\n {report} \n\n Matriz de confusión: \n\n {cf} \n\n Accuracy: \n\n {acc} \n\n')
#Metricas para train
pred = pipline_grad.predict(X_train_over)
report, cf, acc = classification_report(Y_train_over, pred), confusion_matrix(Y_train_over, pred), accuracy_score(Y_train_over, pred)
print(f'Para TRAIN: \n\n Reporte general: \n\n {report} \n\n Matriz de confusión: \n\n {cf} \n\n Accuracy: \n\n {acc}')

Para TEST:

 Reporte general: 

               precision    recall  f1-score   support

           0       0.74      0.96      0.83        26
           1       0.98      0.83      0.90        53

    accuracy                           0.87        79
   macro avg       0.86      0.90      0.87        79
weighted avg       0.90      0.87      0.88        79
 

 Matriz de confusión: 

 [[25  1]
 [ 9 44]] 

 Accuracy: 

 0.8734177215189873 


Para TRAIN: 

 Reporte general: 

               precision    recall  f1-score   support

           0       0.94      0.97      0.95       212
           1       0.97      0.93      0.95       212

    accuracy                           0.95       424
   macro avg       0.95      0.95      0.95       424
weighted avg       0.95      0.95      0.95       424
 

 Matriz de confusión: 

 [[206   6]
 [ 14 198]] 

 Accuracy: 

 0.9528301886792453


En el modelo de gradient hubieron ciertos fallos a la hora de poner los hyperparams ya qeu este estaba presentando un fuerte cao de overfitting teniendo accuracy: 1.0 y datos de test mucho mas bajos por lo cual opte por cambiar ciertos parametros como:

**learning_rate:** Antes: 0.1 | Ahora: 0.01

**Estimators:** Antes: 200  | Ahora: 100

Reduciendo el overfitting y obteniendo buena generalizacion pero mas ajustada que el otro modelo

In [28]:
#Crear validación cruzada 
folds = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipline_kn, X_train_smote, Y_train_smote, cv=folds, scoring='f1')
print('='*50)
print(f'Scores: {scores}')
print('='*50)
print(f'Promedio de scores {np.mean(scores)}')
print('='*50)

Scores: [0.825      0.9        0.94117647 0.98765432 0.93670886]
Promedio de scores 0.9181079304670765


# **Modelo seleccionado**

El modelo seleccionado fue KNeighborsClassifier este modelo nos da mucha mejor generalizacion y buen performance, gradient si bien es un modelo mas robusto , nos otorgó con los parametros sugeridos por el buscador un overfitting el cual lo reducimos agregando un learning rate mas pequeño y bajando la cantidad de estimadores. Sin embargo para mantener una generalización mucho mejor elijo el modelo lineal con la variables mas correlacionadas y lineal  